In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup
import re
import unicodedata
import requests
import os
import datetime

### get data from api

In [2]:
#Global variables
BoosterVersion = []
PayloadMass = []
Orbit = []
LaunchSite = []
Outcome = []
Flights = []
GridFins = []
Reused = []
Legs = []
LandingPad = []
Block = []
ReusedCount = []
Serial = []
Longitude = []
Latitude = []

In [3]:
url = 'https://api.spacexdata.com/v4/launches/past'
response = requests.get(url)
df = pd.json_normalize(response.json())

In [4]:
# Takes the dataset and uses the rocket column to call the API and append the data to the list
def getBoosterVersion(data):
    for rocket in data['rocket']:
       if rocket:
        response = requests.get("https://api.spacexdata.com/v4/rockets/"+str(rocket)).json()
        BoosterVersion.append(response['name'])

In [5]:
# Takes the dataset and uses the launchpad column to call the API and append the data to the list
def getLaunchSite(data):
    for pad in data['launchpad']:
       if pad:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(pad)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

In [6]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
       if load:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

In [7]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])

In [8]:
df.sample()

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
128,None,NaN,False,0.0,5e9d0d95eda69973a809d1ec,True,[],SpaceX's 22nd ISS resupply mission on behalf o...,[],"[5ea6ed2f080df4000697c90b, 608c1a06cf7f3d61526...",...,None,https://www.reddit.com/r/spacex/comments/k2ts1...,[],[https://live.staticflickr.com/65535/512254820...,None,https://youtu.be/QXf9mRWbXDM,QXf9mRWbXDM,https://spaceflightnow.com/2021/06/03/spacex-s...,https://en.wikipedia.org/wiki/SpaceX_CRS-22,NaN


In [9]:
data = df[['rocket', 'payloads', 'launchpad', 'cores', 'flight_number', 'date_utc']]

In [10]:
data = data[data['cores'].map(len)==1]
data = data[data['payloads'].map(len)==1]

In [11]:
data['cores'] = data['cores'].map(lambda x : x[0])
data['payloads'] = data['payloads'].map(lambda x : x[0])

In [12]:
data['date'] = pd.to_datetime(data['date_utc']).dt.date


In [13]:
data = data[data['date'] <= datetime.date(2020, 11, 13)]

In [14]:
getBoosterVersion(data)
getCoreData(data)
getLaunchSite(data)
getPayloadData(data)

In [15]:
launch_dict = {'FlightNumber': list(data['flight_number']),
'Date': list(data['date']),
'BoosterVersion':BoosterVersion,
'PayloadMass':PayloadMass,
'Orbit':Orbit,
'LaunchSite':LaunchSite,
'Outcome':Outcome,
'Flights':Flights,
'GridFins':GridFins,
'Reused':Reused,
'Legs':Legs,
'LandingPad':LandingPad,
'Block':Block,
'ReusedCount':ReusedCount,
'Serial':Serial,
'Longitude': Longitude,
'Latitude': Latitude}

In [81]:
df = pd.DataFrame(launch_dict)

In [83]:
df.sample(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
11,15,2014-07-14,Falcon 9,1316.0,LEO,CCSFS SLC 40,True Ocean,1,False,False,True,None,1.0,0,B1007,-80.577366,28.561857
4,6,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
17,22,2015-04-14,Falcon 9,1898.0,ISS,CCSFS SLC 40,False ASDS,1,True,False,True,5e9e3032383ecb761634e7cb,1.0,0,B1015,-80.577366,28.561857
80,92,2020-03-18,Falcon 9,15600.0,VLEO,KSC LC 39A,False ASDS,5,True,True,True,5e9e3032383ecb6bb234e7ca,5.0,4,B1048,-80.603956,28.608058
82,94,2020-05-30,Falcon 9,9525.0,ISS,KSC LC 39A,True ASDS,1,True,False,True,5e9e3032383ecb6bb234e7ca,5.0,13,B1058,-80.603956,28.608058


### Filter the dataframe to only include `Falcon 9` launches


In [84]:
data_falcon9 = df[df['BoosterVersion']=='Falcon 9']


Now that we have removed some values we should reset the FlgihtNumber column


In [85]:
data_falcon9.loc[:,'FlightNumber'] = list(range(1, data_falcon9.shape[0]+1))
data_falcon9.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
4,1,2010-06-04,Falcon 9,NaN,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0003,-80.577366,28.561857
5,2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0005,-80.577366,28.561857
6,3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B0007,-80.577366,28.561857
7,4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,False,False,False,None,1.0,0,B1003,-120.610829,34.632093
8,5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,False,False,False,None,1.0,0,B1004,-80.577366,28.561857


## Data Wrangling


We can see below that some of the rows are missing values in our dataset.


In [86]:
data_falcon9.isnull().sum()

,0
FlightNumber,0
Date,0
BoosterVersion,0
PayloadMass,5
Orbit,0
LaunchSite,0
Outcome,0
Flights,0
GridFins,0
Reused,0


Before we can continue we must deal with these missing values. The <code>LandingPad</code> column will retain None values to represent when landing pads were not used.


### Task 3: Dealing with Missing Values


Calculate below the mean for the <code>PayloadMass</code> using the <code>.mean()</code>. Then use the mean and the <code>.replace()</code> function to replace `np.nan` values in the data with the mean you calculated.


#### Calculating the mean value of PayloadMass column
#### Replacing the np.nan values with its mean value


In [165]:
mean = data_falcon9['PayloadMass'].mean()
data_falcon9['PayloadMass'].fillna(mean, inplace=True)


<ipython-input-165-1e56d05301c6>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_falcon9['PayloadMass'].fillna(mean, inplace=True)


You should see the number of missing values of the <code>PayLoadMass</code> change to zero.


In [166]:
data_falcon9.isnull().sum()

,0
FlightNumber,0
Date,0
BoosterVersion,0
PayloadMass,0
Orbit,0
LaunchSite,0
Outcome,0
Flights,0
GridFins,0
Reused,0


In [167]:
df = data_falcon9

In [168]:
df.isnull().sum()/len(df)*100

,0
FlightNumber,0.000000
Date,0.000000
BoosterVersion,0.000000
PayloadMass,0.000000
Orbit,0.000000
LaunchSite,0.000000
Outcome,0.000000
Flights,0.000000
GridFins,0.000000
Reused,0.000000


In [169]:
df['LaunchSite'].value_counts()

,count
LaunchSite,
CCSFS SLC 40,55
KSC LC 39A,22
VAFB SLC 4E,13


In [170]:
df['Orbit'].value_counts()

,count
Orbit,
GTO,27
ISS,21
VLEO,14
PO,9
LEO,7
SSO,5
MEO,3
ES-L1,1
HEO,1


In [171]:
landing_outcomes = df['Outcome'].value_counts()

In [172]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


In [173]:
bad_outcomes = set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

In [174]:
landing_class = []
for i in df['Outcome']:
    if i in bad_outcomes:
        landing_class.append(0)
    else:
        landing_class.append(1)
landing_class;

In [175]:
df['Class'] = landing_class

In [176]:
df['Class'].value_counts()

,count
Class,
1,60
0,30


In [177]:
df['Class'].mean()

0.6666666666666666

In [178]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [179]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [180]:
%sql sqlite:///my_data1.db

In [181]:
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

90

In [182]:
%sql DROP TABLE IF EXISTS SPACEXTABLE

 * sqlite:///my_data1.db
Done.


[]

In [183]:
%sql CREATE TABLE SPACEXTABLE AS SELECT * FROM SPACEXTBL WHERE DATE IS NOT NULL

 * sqlite:///my_data1.db
Done.


[]

In [184]:
%sql select distinct LaunchSite from spacextbl

 * sqlite:///my_data1.db
Done.


LaunchSite
CCSFS SLC 40
VAFB SLC 4E
KSC LC 39A


In [185]:
%%sql select * from SPACEXTBL
      where launchsite like 'CCA%'

 * sqlite:///my_data1.db
Done.


FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class


In [186]:
%%sql select * from spacextbl
      where outcome like 'True%'
      order by date
      limit 1

 * sqlite:///my_data1.db
Done.


FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
7,2014-04-18,Falcon 9,2296.0,ISS,CCSFS SLC 40,True Ocean,1,0,0,1,None,1.0,0,B1006,-80.577366,28.5618571,1


In [187]:
%%sql select BoosterVersion from spacextbl
      where outcome like 'True asds' and payloadmass between 4000 and 6000

 * sqlite:///my_data1.db
Done.


BoosterVersion
Falcon 9
Falcon 9
Falcon 9
Falcon 9
Falcon 9
Falcon 9
Falcon 9


In [188]:
%sql select * from spacextbl

 * sqlite:///my_data1.db
Done.


FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
1,2010-06-04,Falcon 9,6123.547647058824,LEO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B0003,-80.577366,28.5618571,0
2,2012-05-22,Falcon 9,525.0,LEO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B0005,-80.577366,28.5618571,0
3,2013-03-01,Falcon 9,677.0,ISS,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B0007,-80.577366,28.5618571,0
4,2013-09-29,Falcon 9,500.0,PO,VAFB SLC 4E,False Ocean,1,0,0,0,None,1.0,0,B1003,-120.610829,34.632093,0
5,2013-12-03,Falcon 9,3170.0,GTO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B1004,-80.577366,28.5618571,0
6,2014-01-06,Falcon 9,3325.0,GTO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B1005,-80.577366,28.5618571,0
7,2014-04-18,Falcon 9,2296.0,ISS,CCSFS SLC 40,True Ocean,1,0,0,1,None,1.0,0,B1006,-80.577366,28.5618571,1
8,2014-07-14,Falcon 9,1316.0,LEO,CCSFS SLC 40,True Ocean,1,0,0,1,None,1.0,0,B1007,-80.577366,28.5618571,1
9,2014-08-05,Falcon 9,4535.0,GTO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B1008,-80.577366,28.5618571,0
10,2014-09-07,Falcon 9,4428.0,GTO,CCSFS SLC 40,None None,1,0,0,0,None,1.0,0,B1011,-80.577366,28.5618571,0


In [189]:
%%sql
select
case when count(outcome) from spacextbl
group by outcome

 * sqlite:///my_data1.db
(sqlite3.OperationalError) near "from": syntax error
[SQL: select 
case when count(outcome) from spacextbl
group by outcome]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
